<a href="https://colab.research.google.com/github/JDA156/Hitss/blob/main/prueba_hitss_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba Hitss

In [14]:
# Configuración para ejecutar Selenium en Google Colab
## No es necesario ejecutar este código si lo hace en Jupyter Notebook u otra configuración local de Python.
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,557 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,296 kB]
Fetched 5,238 kB in 6s (903 kB/s)
Reading package lists... Done
Building dependency tree

In [17]:
!pip install chromedriver-autoinstaller
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
import re
import chromedriver_autoinstaller  # configurar opciones de chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Asegúrese de que la GUI esté desactivada
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # Establezca la ruta a Chromedriver según su configuración
chromedriver_autoinstaller.install()  # establecer la URL de destino
driver = webdriver.Chrome(options=chrome_options)

In [18]:
# ==============================================================================
# SCRIPT DE AUTOMATIZACIÓN - EXTRACCIÓN DE PRODUCTOS
# ==============================================================================
# Este script automatiza la navegación en Mercado Libre y extrae información
# relevante de los productos después de aplicar filtros y ordenar resultados.
# ==============================================================================

import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# --- Configuración Inicial ---
if not os.path.exists("informe_capturas"):
    os.makedirs("informe_capturas")

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('--log-level=3')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
wait = WebDriverWait(driver, 20) # Tiempo de espera para elementos

# --- Función de Reintento de Clic ---
def safe_click_con_scroll(locator, description):
    """
    Intenta hacer scroll hasta el elemento y luego hace clic, reintentando hasta 3 veces.
    """
    for attempt in range(3):
        try:
            element = wait.until(EC.presence_of_element_located(locator))
            # Realiza scroll para asegurar que el elemento esté visible
            driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", element)
            time.sleep(1) # Pequeña pausa para que se complete el scroll

            element_clickable = wait.until(EC.element_to_be_clickable(locator))
            element_clickable.click()

            print(f"Éxito al hacer clic en: '{description}'")
            return
        except Exception as e:
            print(f"Intento {attempt + 1} fallido para '{description}': {type(e)._name_}")
            if attempt < 2:
                print("Reintentando en 2 segundos...")
                time.sleep(2)
            else:
                print(f"FALLO DEFINITIVO: No se pudo hacer clic en '{description}' después de 3 intentos.")
                raise

# --- Inicio de la Automatización ---
try:
    driver.get('https://www.mercadolibre.com/')

    # Aceptar cookies
    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Aceptar cookies')]"))).click()
        print("Cookies aceptadas.")
        time.sleep(1)
    except TimeoutException:
        print("No se encontró el banner de cookies.")

    # 1. Seleccionar país
    safe_click_con_scroll((By.XPATH, '//*[@id="MX"]'), "País México")
    time.sleep(2)
    driver.save_screenshot('informe_capturas/01_pais_seleccionado.png')

    # 2. Buscar "playstation 5"
    search_bar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cb1-edit"]')))
    search_bar.send_keys('playstation 5')
    safe_click_con_scroll((By.XPATH, '/html/body/header/div/div[2]/form/button'), "Botón de búsqueda")
    time.sleep(3)
    driver.save_screenshot('informe_capturas/02_resultados_busqueda.png')

    # 3. Aplicar filtro "Nuevo"
    print("Aplicando filtro 'Nuevo'...")
    safe_click_con_scroll((By.CSS_SELECTOR, '#root-app > div > div.ui-search-main.ui-search-main--only-products.ui-search-main--with-topkeywords > aside > section.ui-search-filter-groups > div:nth-child(5) > ul > li:nth-child(1) > a > span.ui-search-filter-name'), "Filtro Nuevo")
    time.sleep(3)
    driver.save_screenshot('informe_capturas/03_filtro_nuevos.png')

    # 4. Aplicar filtro de ubicación "CDMX"
    print("Aplicando filtro de ubicación 'CDMX'...")
    safe_click_con_scroll((By.CSS_SELECTOR, '#root-app > div > div.ui-search-main.ui-search-main--only-products.ui-search-main--with-topkeywords > aside > section.ui-search-filter-groups > div:nth-child(10) > ul > li:nth-child(1) > a > span.ui-search-filter-name'), "Filtro Ubicación CDMX")
    time.sleep(3)
    driver.save_screenshot('informe_capturas/04_filtro_ubicacion.png')

    # 5. Ordenar por precio "Más Relevantes" (usando un selector más genérico para el desplegable)
    print("Ordenando por precio...")
    # Se usa un selector más general para el desplegable de ordenar, ya que el ID puede variar.
    # Se busca el elemento que contenga el texto "Más relevantes" que es el valor por defecto
    safe_click_con_scroll((By.XPATH, '//*[@id=":R1b56ie:-display-values"]'), "Desplegable para ordenar")
    time.sleep(2)
    # Se selecciona la opción "Menor precio" buscando su texto visible
    safe_click_con_scroll((By.ID, ":R1b56ie:-menu-list-option-price_asc"), "Opción 'Menor precio'")
    time.sleep(3)
    driver.save_screenshot('informe_capturas/05_ordenado_menor_precio.png')

    # 6. Extraer los datos de los productos
    print("Extrayendo información de los 5 primeros productos...")
    # Se utiliza un selector más robusto para los ítems que contiene los productos.
    items = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//ol[contains(@class, 'ui-search-layout')]/li[contains(@class, 'ui-search-layout__item')]")))

    print("\n" + "="*80)
    print("                      INFORME FINAL: 5 PRODUCTOS PLAYSTATION 5")
    print("="*80)

    products_extracted = 0
    for i, item in enumerate(items):
        if products_extracted >= 5:
            break

        try:
            brand_element = item.find_elements(By.CLASS_NAME, "poly-component__brand")
            brand = brand_element[0].text if brand_element else "N/A"

            name = item.find_element(By.CLASS_NAME, "poly-component__title").text

            current_price_fraction = item.find_element(By.XPATH, ".//div[contains(@class, 'poly-price_current')]//span[contains(@class, 'andes-money-amount_fraction')]").text
            current_price_cents_elements = item.find_elements(By.XPATH, ".//div[contains(@class, 'poly-price_current')]//span[contains(@class, 'andes-money-amount_cents')]")
            current_price_cents = current_price_cents_elements[0].text if current_price_cents_elements else "00"
            current_price = f"${current_price_fraction}.{current_price_cents}"

            # Intentar obtener precio anterior
            previous_price_element = item.find_elements(By.XPATH, ".//s[contains(@class, 'andes-money-amount--previous')]//span[contains(@class, 'andes-money-amount__fraction')]")
            previous_price = f"${previous_price_element[0].text}" if previous_price_element else "N/A"

            # Intentar obtener descuento
            discount_element = item.find_elements(By.CLASS_NAME, "andes-money-amount__discount")
            discount = discount_element[0].text if discount_element else "N/A"

            # Calificaciones
            rating_element = item.find_elements(By.CLASS_NAME, "poly-reviews__rating")
            rating = rating_element[0].text if rating_element else "N/A"

            total_reviews_element = item.find_elements(By.CLASS_NAME, "poly-reviews__total")
            total_reviews = total_reviews_element[0].text.strip('()') if total_reviews_element else "N/A"

            print(f"Producto {products_extracted+1}:")
            print(f"  Marca: {brand}")
            print(f"  Nombre: {name}")
            print(f"  Precio Actual: {current_price}")
            print(f"  Precio Anterior: {previous_price}")
            print(f"  Descuento: {discount}")
            print(f"  Calificación Promedio: {rating}")
            print(f"  Número de Calificaciones: {total_reviews}\n")
            products_extracted += 1

        except NoSuchElementException as e:
            print(f"No se pudo extraer toda la información para el producto {i+1}. Algunos elementos no se encontraron. Error: {e}\n")
        except Exception as e:
            print(f"Ocurrió un error inesperado al procesar el producto {i+1}: {e}\n")

    print("="*80)

except Exception as e:
    print("\n-------------------------------------------------------------")
    print("                     EL SCRIPT HA FALLADO")
    print("-------------------------------------------------------------")
    print(f"\nEl error fue de tipo: {type(e)._name_}")
    error_filename = 'informe_capturas/error_screenshot.png'
    driver.save_screenshot(error_filename)
    print(f"Se guardó una captura del error en '{error_filename}'.")
    print("\nVerifica las rutas de los selectores o la estructura de la página de Mercado Libre.")

finally:
    if 'driver' in locals() and driver:
        driver.quit()
    print("\nPrueba finalizada.")

No se encontró el banner de cookies.
Éxito al hacer clic en: 'País México'
Éxito al hacer clic en: 'Botón de búsqueda'
Aplicando filtro 'Nuevo'...
Éxito al hacer clic en: 'Filtro Nuevo'
Aplicando filtro de ubicación 'CDMX'...
Éxito al hacer clic en: 'Filtro Ubicación CDMX'
Ordenando por precio...
Éxito al hacer clic en: 'Desplegable para ordenar'
Éxito al hacer clic en: 'Opción 'Menor precio''
Extrayendo información de los 5 primeros productos...

                      INFORME FINAL: 5 PRODUCTOS PLAYSTATION 5
No se pudo extraer toda la información para el producto 1. Algunos elementos no se encontraron. Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class, 'poly-price_current')]//span[contains(@class, 'andes-money-amount_fraction')]"}
  (Session info: chrome=137.0.7151.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace